# VacationPy
----

#### Note
* Keep an eye on your API usage. Use https://developers.google.com/maps/reporting/gmp-reporting as reference for how to monitor your usage and billing.

* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [38]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os

# Import API key
from api_keys import g_key

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [39]:
# Loading in the prior weather dataset via Pandas' read_csv method.  Not sure when to know
# if/when we should be specifying encounding parameters? Explicitly stated it based on the
# class activity.
#!  TA / Tutor Question

csv_file = "../WeatherPy/Export_data/consolidated_weather_data.csv"
initial_df = pd.read_csv(csv_file, encoding="ISO-8859-1")
initial_df.head()

,Unnamed: 0,City,Lat,lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,Barrow,71.2906,-156.7887,255.15,71,90,4.63,US,1612631204
1,1,Great Falls,47.5002,-111.3008,255.37,78,90,5.14,US,1612631124
2,2,Narsaq,60.9167,-46.0500,274.15,25,1,10.80,GL,1612631063
3,3,Atuona,-9.8000,-139.0333,299.91,77,57,10.02,PF,1612631209
4,4,Castro,-24.7911,-50.0119,297.74,50,0,3.33,BR,1612631210


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [40]:
##  Referencing Activity 3-7 for guidance on this part ##

#  Step 1:  Connect to Google Map's API, we'll be using the specific Places API here.
gmaps.configure(api_key=g_key)

#  Step 2:  Set locations using Lat & Lng
locations = initial_df[["Lat", "lng"]]

#  Step 3:  Set heatmap values and convert to float type
    # Fill NaN values and convert to float
humid_value = initial_df["Humidity"].astype(float)

#  Step 4:  Plot the baseline map layer
    # Plot Heatmap
fig = gmaps.figure()

#  Step 5:  Initialize the heatmap overlay, incorporating our locale and heatmap specs above
    # Create heat layer
heat_layer = gmaps.heatmap_layer(locations, weights=humid_value, 
                                 dissipating=False, max_intensity=10,
                                 point_radius=1)

#  Step 6:  Add the newly-created heatmap overlay to our basemap
    # Add layer
fig.add_layer(heat_layer)

# Display figure
fig

Figure(layout=FigureLayout(height='420px'))

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [41]:
#! I really wanted to consolidate all of these conditionals into one nice line, howevever I 
#  kept getting a flaat error.  TA / Tutor question:  What was I doing wrong?.

ideal_vacay_localesA = initial_df[initial_df["Max Temp"] < 299.81]
ideal_vacay_localesB =  ideal_vacay_localesA[ideal_vacay_localesA["Max Temp"] > 294.26]
ideal_vacay_localesC =  ideal_vacay_localesB[ideal_vacay_localesB["Wind Speed"] < 10]
ideal_vacay_localesD =  ideal_vacay_localesC[ideal_vacay_localesC["Cloudiness"] == 0]
ideal_vacay_localesD.dropna(how='any')
ideal_vacay_localesD.count()

Unnamed: 0    17
City          17
Lat           17
lng           17
Max Temp      17
Humidity      17
Cloudiness    17
Wind Speed    17
Country       17
Date          17
dtype: int64

### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [42]:
hotel_df = ideal_vacay_localesD
hotel_df["Hotel"] = ""
hotel_df.head()

C:\Users\simon\anaconda3\envs\PythonData\lib\site-packages\ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


,Unnamed: 0,City,Lat,lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date,Hotel
4,4,Castro,-24.7911,-50.0119,297.74,50,0,3.33,BR,1612631210,
46,46,Ancud,-41.8697,-73.8203,298.15,44,0,3.60,CL,1612631035,
66,66,Kruisfontein,-34.0033,24.7314,295.37,80,0,0.89,ZA,1612631309,
187,187,MÄngrol,21.1167,70.1167,294.52,48,0,2.81,IN,1612631505,
212,212,Mocorito,25.4833,-107.9167,298.83,27,0,0.93,MX,1612631545,


In [48]:
params = {
    "radius": 50000,
    "types": "hotel",
    "keyword": "hotel",
    "key": g_key
}

# Use the lat/lng we recovered to identify airports
for index, row in hotel_df.iterrows():
    # get lat, lng from df
    lat = row["Lat"]
    lng = row["lng"]

    # change location each iteration while leaving original params in place
    params["location"] = f"{lat},{lng}"

    # Use the search term: "International Airport" and our lat/lng
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

    # make request and print url
    nearby_hotel = requests.get(base_url, params=params)
    
#     print the name_address url, avoid doing for public github repos in order to avoid exposing key
#     print(name_address.url)

    # convert to json
    nearby_hotel = nearby_hotel.json()
    # print(json.dumps(name_address, indent=4, sort_keys=True))

    # Since some data may be missing we incorporate a try-except to skip any that are missing a data point.
    try:
        hotel_df.loc[index, "Hotel"] = nearby_hotel["results"][0]["name"]
#         cities_pd.loc[index, "Airport Address"] = name_address["results"][0]["vicinity"]
#         cities_pd.loc[index, "Airport Rating"] = name_address["results"][0]["rating"]
    except (KeyError, IndexError):
        print("Missing field/result... skipping.")

hotel_df.head()

C:\Users\simon\anaconda3\envs\PythonData\lib\site-packages\pandas\core\indexing.py:1763: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  isetter(loc, value)


,Unnamed: 0,City,Lat,lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date,Hotel
4,4,Castro,-24.7911,-50.0119,297.74,50,0,3.33,BR,1612631210,Hotel Borgen
46,46,Ancud,-41.8697,-73.8203,298.15,44,0,3.60,CL,1612631035,Panamericana Hotel Ancud
66,66,Kruisfontein,-34.0033,24.7314,295.37,80,0,0.89,ZA,1612631309,Savoy Hotel
187,187,MÄngrol,21.1167,70.1167,294.52,48,0,2.81,IN,1612631505,Aaranya Resort - Sasan Gir
212,212,Mocorito,25.4833,-107.9167,298.83,27,0,0.93,MX,1612631545,La Cuarteria Hotel Boutique


In [50]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Lat", "lng"]]

In [ ]:
#! TA / Tutor Question:  Couldn't get the hotel_info layer to display, I kept getting these errors:
#  Error 1: When attempting
#      fig.add_layer(heat_layer)
#      fig.add_layer(hotel_info)

#      TraitError: The 'None' trait of a Map instance must be a Widget, but a value of class 'list' wh
#
#  Error 2:  When attempting
#      fig.add_layer(hotel_info(heat_layer))
#
#      TypeError: 'list' object is not callable


# Add marker layer ontop of heat map


# Display figure


In [61]:
##  Referencing Activity 3-7 for guidance on this part ##

#  Step 1:  Connect to Google Map's API, we'll be using the specific Places API here.
gmaps.configure(api_key=g_key)

#  Step 2:  Set locations using Lat & Lng
locations = hotel_df[["Lat", "lng"]]

#  Step 3:  Set heatmap values and convert to float type
    # Fill NaN values and convert to float
humid_value = hotel_df["Humidity"].astype(float)

#  Step 4:  Plot the baseline map layer
    # Plot Heatmap
fig = gmaps.figure()

#  Step 5:  Initialize the heatmap overlay, incorporating our locale and heatmap specs above
    # Create heat layer
heat_layer = gmaps.heatmap_layer(locations, weights=humid_value, 
                                 dissipating=False, max_intensity=10,
                                 point_radius=1)

#  Step 6:  Add the newly-created heatmap overlay to our basemap
    # Add layer
fig.add_layer(heat_layer)


# Display figure
fig

Figure(layout=FigureLayout(height='420px'))